In [1]:
import pandas as pd
import cvxpy as cp

# File path will be different on another device and must be replaced with the correct file path.
students = pd.read_csv("C:/Users/Morgan Eads/Documents/ISE 3230/Sample Data (for scheduling) - Sheet1.csv")
print(students.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               60 non-null     int64 
 1   First Name       60 non-null     object
 2   Last Name        60 non-null     object
 3   Gender           60 non-null     object
 4   Age              60 non-null     int64 
 5   Times Available  60 non-null     object
dtypes: int64(2), object(4)
memory usage: 2.9+ KB
None


In [2]:
students['Age'].value_counts()

Age
9     13
12    12
11    11
13     9
8      8
10     7
Name: count, dtype: int64

In [ ]:
# TODO: Figure out how to create a new column that separates age into Elem & MS.